In [1]:
%load_ext tensorboard

In [2]:
from dataclasses import dataclass
from datetime import datetime
from enum import Enum
import json
import logging
import math
from typing import Tuple, List, Type, Union

import chemprop
import numpy as np
import pandas as pd
import pickle
import pytorch_lightning as tl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from rdkit import Chem
import scipy as sp
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, max_error
from sklearn.model_selection import KFold
import torch
from torch import Tensor
from torch.nn import ReLU, Linear, MSELoss, Dropout
from torch.nn.functional import log_softmax, relu, dropout
from torch.optim import AdamW
from torchmetrics.functional import pearson_corrcoef
from torch_geometric.data import Data, Batch, InMemoryDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import (
    Sequential,
    MessagePassing, GCNConv, GATConv, GATv2Conv, GINConv,
    Aggregation, global_mean_pool, global_max_pool, global_add_pool
)

C:\Users\MrMil\miniconda3\envs\automl-drug-discovery\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
def root_mean_squared_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

def correlation_squared_score(y_true, y_pred):
    return np.power(pearson_corrcoef(y_pred, y_true), 2)

def correlation_score(y_true, y_pred):
    return pearson_corrcoef(y_pred, y_true)

In [4]:
def generate_experiment_path(dataset_name, using_embeddings):
    data_used = 'SD_DR' if using_embeddings else 'DR'
    return f"{dataset_name}/{data_used}"

def generate_run_name():
    return datetime.now().strftime("%m-%d-%Y_%H-%M-%S")

In [5]:
def save_hyper_parameters(parameters, architecture, run_dir):
    filepath = run_dir + '/' + 'parameters.txt'
    with open(filepath, 'w') as out:
        out.write(str(architecture))
        out.write('\n')
        out.write(str(parameters))

In [6]:
def save_architecture(architecture, run_dir):
    filepath = run_dir + '/' + 'architecture.pkl'
    with open(filepath, 'wb') as out:
        pickle.dump(architecture, out)
        
def load_architecture(run_dir):
    filepath = run_dir + '/' + 'architecture.pkl'
    with open(filepath, 'rb') as file:
        return pickle.load(file)

In [7]:
def save_k_fold_metrics(fold_metrics, run_dir):
    filepath = run_dir + '/' + 'results.json'
    stacked_metrics = {name: np.array([fold[name] for fold in fold_metrics]) for name in fold_metrics[0]}
    means = {name: float(np.mean(metrics)) for name, metrics in stacked_metrics.items()}
    variances = {name: float(np.var(metrics)) for name, metrics in stacked_metrics.items()}
    metrics = {name: {'mean': means[name], 'variance': variances[name]} for name in stacked_metrics}
    with open(filepath, 'w') as out:
        json.dump(metrics, out)

# Data Prep
Each dataset has the following columns: CID, SD, SD Z-score, DR, XC50, activity, neut-smiles, num. atoms and max atomic num. 

For the base of the project I will only be using the DR value and the neut-smiles representation. For any compound with a non-null DR value, the activity is 'Active'.

In [8]:
def get_connectivity(mol):
    conns = []
    b2a = mol.b2a
    a2b = mol.a2b
    for aI, bonds in enumerate(a2b):
        neighbours = [(b2a[bI], aI) for bI in bonds]
        conns.extend(neighbours)
    return conns

def read_data(filepath: str) -> pd.DataFrame:
    return pd.read_csv(filepath)

def process_data(df: pd.DataFrame) -> Tuple[List[Tensor]]:
    smiles = df['neut-smiles']
    mols = [chemprop.features.featurization.MolGraph(s) for s in smiles]
    xs = [Tensor(m.f_atoms) for m in mols]
    conns = [get_connectivity(m) for m in mols]
    edge_indexes = [torch.tensor(conn, dtype=torch.long).T.contiguous() for conn in conns]
    ys = Tensor(df['DR'].values)
    return [Data(x=x, edge_index=index, y=y) for x, index, y in zip(xs, edge_indexes, ys)]

In [9]:
class DRDataset(InMemoryDataset):
    def __init__(self, root):
        super().__init__(root)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    def __get__(self, idx):
        return self.get(idx)

    @property
    def raw_file_names(self):
        return [DATAFILE]

    @property
    def processed_file_names(self):
        return ['processed_data.pt']

    def process(self):
        df = read_data(self.root + '\\' + self.raw_file_names[0])
        dr_df = df[df['DR'].notnull()]
        data_list = process_data(dr_df)
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [10]:
def split_dataset(dataset, ratio):
    num_train_samples = int(ratio * len(dataset))
    training_dataset = dataset.index_select(slice(num_train_samples))
    validation_dataset = dataset.index_select(slice(num_train_samples, None))
    return training_dataset, validation_dataset

In [11]:
def k_folds(dataset, k, seed):
    kfold = KFold(n_splits=k, shuffle=True, random_state=seed)
    for train_index, val_index in kfold.split(dataset):
        train_dataset = dataset.index_select(train_index.tolist())
        val_dataset = dataset.index_select(val_index.tolist())
        yield train_dataset, val_dataset

In [12]:
class GNNLayer(Enum):
    GCN = GCNConv
    GIN = GINConv
    GAT = GATConv
    GATv2 = GATv2Conv

class ActivationFunction(Enum):
    ReLU = ReLU
        
class PoolingFunction(Enum):
     MEAN = global_mean_pool
     MAX = global_max_pool
     ADD = global_add_pool
        
@dataclass
class ModelArchitecture:
    layer_types: List[GNNLayer]
    features: List[int]
    activation_funcs: List[ActivationFunction]
    pool_func: PoolingFunction
    
    def __str__(self):
        return f"""\
        ModelArchitecture(
            Layers: [{', '.join([layer.name for layer in self.layer_types])}]
            Features: {self.features}
            Activation Functions: [{', '.join([func.name if func is not None else 'None' for func in self.activation_funcs])}]
            Pool Function: {self.pool_func.__name__}
        )
        """

In [13]:
def construct_sequential_model(arch: ModelArchitecture) -> Sequential:
    global_inputs = "x, edge_index, batch"
    layers = []
    for layer_type, num_in, num_out, activation in zip(
        arch.layer_types,
        arch.features[:-1],
        arch.features[1:],
        arch.activation_funcs
    ):
        layers.append((layer_type.value(num_in, num_out), "x, edge_index -> x"))
        if activation is not None:
            layers.append(activation.value(inplace=True))
    layers.append((arch.pool_func, "x, batch -> x"))
    return Sequential(global_inputs, layers)

In [28]:
class LitGNN(tl.LightningModule):
    def __init__(self, architecture, metrics, batch_size):
        super().__init__()
        self.gnn = construct_sequential_model(architecture)
        self.loss = MSELoss()
        self.metrics = metrics
        self.batch_size = batch_size
        
    def training_step(self, data, idx):
        pred, loss = self._eval(data)
        self._report_loss(loss, prefix='train')
        metrics = self._calculate_metrics(data.y, pred)
        metrics = {'train_' + name: val for name, val in metrics.items()}
        self.log_dict(metrics, logger=True, on_step=True, batch_size=data.y.shape[0])
        return loss

    def validation_step(self, data, idx):
        pred, loss = self._eval(data)
        self._report_loss(loss, prefix='val')
        metrics = self._calculate_metrics(data.y, pred)
        self.log_dict(metrics, logger=True, on_step=False, on_epoch=True, batch_size=data.y.shape[0])
        return loss

    def test_step(self, data, idx):
        pred = self.gnn(data.x, data.edge_index, data.batch)
        return data.y, pred
    
    def test_epoch_end(self, outputs):
        ys, preds = [torch.concat(arr, axis=0) for arr in zip(*outputs)]
        losses = self.loss(preds, ys.reshape(-1, 1))
        metrics = self._calculate_metrics(ys, preds)
        metrics['loss'] = losses
        avg_metrics = {name: metric.mean() for name, metric in metrics.items()}
        self.log_dict(avg_metrics, logger=False, batch_size=self.batch_size)
        self._plot_residuals(ys, preds)

    def configure_optimizers(self):
        optimiser = AdamW(self.gnn.parameters())
        return optimiser
    
    def _eval(self, data):
        pred = self.gnn(data.x, data.edge_index, data.batch)
        loss = self.loss(pred, data.y.reshape(-1, 1))
        return pred, loss
    
    def _report_loss(self, loss, prefix):
        self.log('loss/' + prefix, loss, logger=True, batch_size=self.batch_size)
        
    def _calculate_metrics(self, y, pred):
        local_y, local_pred = y.cpu(), pred.detach().cpu().flatten()
        metrics = {name: metric(y_true=local_y, y_pred=local_pred) for name, metric in self.metrics.items()}
        sanitised_metrics = {name: 0 if val == math.nan else val for name, val in metrics.items()}
        return sanitised_metrics
    
    def _plot_residuals(self, ys, preds):
        residuals = ys.reshape(-1, 1) - preds
        plt.scatter(ys.cpu(), residuals.cpu())
        plt.xlabel("Label")
        plt.ylabel("Residuals")
        plt.savefig(self.logger.log_dir + '/residuals.png')
        plt.clf()
        

In [15]:
@dataclass
class HyperParameters:
    random_seed: int
    k_folds: int
    train_test_split: float
    batch_size: int
    early_stop_patience: int
    early_stop_min_delta: float
    lr: float
    max_epochs: int

# Run trials

In [29]:
DATASET_NAME = 'AID504329'
DATA_DIR = f'../data/{DATASET_NAME}/'
LOG_DIR = '../logs/'
DATAFILE = 'SD.csv'
EXPERIMENT_DIR = LOG_DIR + generate_experiment_path(DATASET_NAME, False)

NUM_WORKERS = 0

DEFAULT_PARAMETERS = HyperParameters(
    random_seed=1424,
    k_folds=4,
    train_test_split=0.8,
    batch_size=32,
    early_stop_patience=10,
    early_stop_min_delta=0.01,
    lr=0.00003,
    max_epochs=100
)

DEFAULT_METRICS = {
    'mae': mean_absolute_error,
    'rmse': root_mean_squared_error,
    'r2': correlation_squared_score,
    'max_error': max_error,
    'r': correlation_score
}

In [30]:
gcn_architecture = ModelArchitecture(
    layer_types=[GNNLayer.GCN, GNNLayer.GCN, GNNLayer.GCN],
    features=[133, 64, 16, 1],
    activation_funcs=[ActivationFunction.ReLU, ActivationFunction.ReLU, None],
    pool_func=PoolingFunction.MEAN
)

gat_architecture = ModelArchitecture(
    layer_types=[GNNLayer.GAT, GNNLayer.GAT, GNNLayer.GAT],
    features=[133, 64, 16, 1],
    activation_funcs=[ActivationFunction.ReLU, ActivationFunction.ReLU, None],
    pool_func=PoolingFunction.MEAN
)

gatv2_architecture = ModelArchitecture(
    layer_types=[GNNLayer.GATv2, GNNLayer.GATv2, GNNLayer.GATv2],
    features=[133, 64, 16, 1],
    activation_funcs=[ActivationFunction.ReLU, ActivationFunction.ReLU, None],
    pool_func=PoolingFunction.MEAN
)

In [31]:
def run_experiment(architecture, params: HyperParameters):
    tl.seed_everything(params.random_seed, workers=True)

    run_dir = EXPERIMENT_DIR + '/' + generate_run_name()

    dataset = DRDataset(root=DATA_DIR)
    dataset.shuffle()
    training_dataset, test_dataset = split_dataset(dataset, params.train_test_split)
    test_dataloader = DataLoader(test_dataset, batch_size=params.batch_size, num_workers=NUM_WORKERS)
    fold_metrics = []
    for train_fold, val_fold in k_folds(training_dataset, params.k_folds, params.random_seed):
        training_dataloader = DataLoader(train_fold, batch_size=params.batch_size, shuffle=True, num_workers=NUM_WORKERS)
        validation_dataloader = DataLoader(val_fold, batch_size=params.batch_size, num_workers=NUM_WORKERS)

        model = LitGNN(architecture, DEFAULT_METRICS, params.batch_size)

        checkpoint_callback = ModelCheckpoint(
            monitor='loss/val',
            mode='min',
            filename='{epoch:02d}-{loss/val:.2f}',
            save_top_k=2,
        )

        early_stop_callback = EarlyStopping(
            monitor='loss/val',
            mode='min',
            patience=params.early_stop_patience,
            min_delta=params.early_stop_min_delta
        )

        trainer = tl.Trainer(
            default_root_dir=run_dir,
            deterministic=True,
            accelerator='gpu',
            devices=1,
            log_every_n_steps=1,
            max_epochs=params.max_epochs,
            #callbacks=[checkpoint_callback, early_stop_callback],
            enable_progress_bar=False,
        )

        trainer.fit(
            model,
            training_dataloader,
            validation_dataloader,
        )

        trainer.test(ckpt_path='best', dataloaders=test_dataloader)
        fold_metrics.append(trainer.callback_metrics)

    save_k_fold_metrics(fold_metrics, run_dir)
    save_architecture(architecture, run_dir)
    save_hyper_parameters(params, architecture, run_dir)

In [32]:
run_experiment(gcn_architecture, DEFAULT_PARAMETERS)

INFO:lightning_lite.utilities.seed:Global seed set to 1424
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type              | Params
-------------------------------------------
0 | gnn  | Sequential_ff6297 | 9.6 K 
1 | loss | MSELoss           | 0     
-------------------------------------------
9.6 K     Trainable params
0         Non-trainable params
9.6 K     Total params
0.039     Total estimated model params size (MB)
C:\Users\MrMil\miniconda3\envs\automl-drug-discovery\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: Th

<Figure size 640x480 with 0 Axes>

In [ ]:
for seed in [5, 546, 985]:
    for architecture in [gcn_architecture, gat_architecture, gatv2_architecture]:
        DEFAULT_PARAMETERS.random_seed = seed
        run_experiment(architecture, DEFAULT_PARAMETERS)

In [33]:
%tensorboard --logdir={EXPERIMENT_DIR}

Reusing TensorBoard on port 6006 (pid 21916), started 0:11:17 ago. (Use '!kill 21916' to kill it.)

In [ ]:
dataset = DRDataset(root=DATA_DIR)
dataset.shuffle()
training_dataset, test_dataset = split_dataset(dataset, TRAIN_TEST_SPLIT)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

def objective(parameters):
    for train_fold, val_fold in k_folds(training_dataset, K_FOLDS):
        training_dataloader = DataLoader(train_fold, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
        validation_dataloader = DataLoader(val_fold, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)
        
        model = LitGNN(architecture, DEFAULT_METRICS)

        early_stop_callback = EarlyStopping(
            monitor='loss/val',
            mode='min',
            patience=3,
        )

        trainer = tl.Trainer(
            default_root_dir=run_dir,
            deterministic=True,
            log_every_n_steps=1,
            max_epochs=MAX_EPOCHS,
            callbacks=[checkpoint_callback, early_stop_callback],
            enable_progress_bar=False
        )

        trainer.fit(
            model,
            training_dataloader,
            validation_dataloader,
        )
        
        return {'loss': trainer.callback_metrics['val/loss'], 'status': hp.STATUS_OK}

_options = [
    {
        'num_layers': num_layers,
        'layer_types': [hp.choice(str(i), GNNLayers.) i in range(num_layers)]
        'features': [hp.choice(str(i), [layer for layer in GNNLayers]) i in range(num_layers)]
    }
    for num_layers in range(2, 5)
]

space = hp.choice('architecture', _options)

best = hp.fmin(
    fn=objective,
    space=search_space,
    algo=hp.tpe.suggest,
    max_evals=100
)


In [199]:
results=json.loads(input())
print()
result_arr = [str(results[name][measure]) for name in ['loss'] + list(DEFAULT_METRICS.keys()) for measure in ['mean', 'variance']]
print("\t".join(result_arr)) 

 {"loss": {"mean": 0.1427062451839447, "variance": 5.5452968808822334e-05}, "mae": {"mean": 0.3190796375274658, "variance": 6.943024345673621e-05}, "rmse": {"mean": 0.3752123713493347, "variance": 9.34278141357936e-05}, "r2": {"mean": -1.9244879484176636, "variance": 0.022452671080827713}, "max_error": {"mean": 0.7781556844711304, "variance": 0.0005830924492329359}}



0.1427062451839447	5.5452968808822334e-05	0.3190796375274658	6.943024345673621e-05	0.3752123713493347	9.34278141357936e-05	-1.9244879484176636	0.022452671080827713	0.7781556844711304	0.0005830924492329359
